## LZW : [Теория](https://neerc.ifmo.ru/wiki/index.php?title=%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC_LZW)

In [1]:
import math

def log2_ceil(n: int):
    return math.ceil(math.log2(n)) 


def pretty_print(n: int, size: int):
    return format(n, f'0{log2_ceil(size)}b')


def get_compressed(code: list, alphabet_size: int):
    data = []
    for item in code:
        data.append(pretty_print(item, alphabet_size))
        alphabet_size += 1
    return data

In [2]:
class Trie():
    def __init__(self):
        self.nodes = {}
        self.leaf = False
        self.code = None
    
    def insert(self, word: str, code: int) -> None:
        current_trie = self
        for char in word:
            if char not in current_trie.nodes:
                current_trie.nodes[char] = Trie()
            current_trie = current_trie.nodes[char]
        current_trie.leaf = True
        current_trie.code = code
    
    def find(self, word: str):
        current_trie = self
        for char in word:
            if char not in current_trie.nodes:
                #return False, None
                return None
            current_trie = current_trie.nodes[char]
        #return current_trie.leaf, current_trie.code
        return current_trie.code


In [3]:
class Lzw():
    def __init__(self, end_symbol: str):
        self.end_symbol = end_symbol
    
    
    def encode(self, alphabet: list, word: str):
        root = Trie()
        
        i = 0
        for char in alphabet:
            root.insert(char, i)
            i += 1

        compressed_data = []
        X = word[0]
        for Y in word[1:]:
            XY = X + Y
            if Y == self.end_symbol:
                compressed_data.append(root.find(X))
            else:
                if root.find(XY) is None:
                    compressed_data.append(root.find(X))

                    root.insert(XY, i)
                    i += 1

                    X = Y
                else:
                    X = XY
        return {
            'data': compressed_data,
            'data bits': get_compressed(compressed_data, len(alphabet)),
        }

    def decode(self, alphabet: list, word: list):##str!!
        root = Trie()

        i = '0'
        for char in alphabet:
            root.insert(i, char)
            i = chr(ord(i) + 1)
        compressed_data = []
        
        X = word[0]
        compressed_data.append(root.find(word[0]))
        for Y in word[1:]:
            if Y == self.end_symbol:
                break

            if root.find(Y) is not None:     
                root.insert(i, root.find(X) + root.find(Y)[0])
                compressed_data.append(root.find(Y))
            else:
                root.insert(i, root.find(X) + root.find(X)[0])
                compressed_data.append(root.find(i))
            
            X = Y
            i = chr(ord(i) + 1)
        return compressed_data

In [7]:
# main test

ALPHABET = ['a', 'b', 'c', 'd', 'e']
END_SYMBOL = '#'
WORD = 'abacabadabacabae'
ENCODED_WORD = '01025039864'
ENCODED_WORD_BITS = '0000010000100101000000111001100001100100'

lzw = Lzw(END_SYMBOL)

encoded = lzw.encode(ALPHABET, WORD + END_SYMBOL)
assert(
    ''.join([str(i) for i in encoded['data']]) == ENCODED_WORD
)

assert(
    ''.join([str(i) for i in encoded['data bits']]) == ENCODED_WORD_BITS
)

decoded = lzw.decode(ALPHABET, ENCODED_WORD + END_SYMBOL)
assert(''.join(decoded) ==  WORD)


In [6]:
# failed test
WORD = '011011100001010100010101010'
ALPHABET = ['0', '1']
END_SYMBOL = '#'

lzw = Lzw(END_SYMBOL)
print(WORD)

encoded = lzw.encode(ALPHABET, WORD + END_SYMBOL)
#print(encoded['data'])


encoded = ''.join(list(map(str, encoded['data'])))
print(encoded, '\n')


decoded = lzw.decode(ALPHABET, encoded + END_SYMBOL)
#print(decoded)
decoded = ''.join(decoded)
print(decoded, '\n')

print(WORD == decoded)

011011100001010100010101010
011230729474139 

011011100001010100010111010 

False
